## METHOD 1 : Using Beautiful Soup

In [35]:
# importing the libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [36]:
#Lets define the URL

url="https://www.worldometers.info/coronavirus/"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "lxml")
#print(soup.prettify()) # print the parsed data of html

In [37]:
#we pick the id of the table we want to scrape and extract HTML code for that particular table only
covid_table = soup.find("table", attrs={"id": "main_table_countries_today"})

In [38]:
#the head will form our columns
head = covid_table.thead.find_all("tr") 
head #the headers are contained in this HTML code

[<tr>
 <th width="1%">#</th>
 <th width="100">Country,<br/>Other</th>
 <th width="20">Total<br/>Cases</th>
 <th width="30">New<br/>Cases</th>
 <th width="30">Total<br/>Deaths</th>
 <th width="30">New<br/>Deaths</th>
 <th width="30">Total<br/>Recovered</th>
 <th width="30">New<br/>Recovered</th>
 <th width="30">Active<br/>Cases</th>
 <th width="30">Serious,<br/>Critical</th>
 <th width="30">Tot Cases/<br/>1M pop</th>
 <th width="30">Deaths/<br/>1M pop</th>
 <th width="30">Total<br/>Tests</th>
 <th width="30">Tests/<br/>
 <nobr>1M pop</nobr>
 </th>
 <th width="30">Population</th>
 <th style="display:none" width="30">Continent</th>
 <th width="30">1 Case<br/>every X ppl</th><th width="30">1 Death<br/>every X ppl</th><th width="30">1 Test<br/>every X ppl</th>
 <th width="30">New Cases/1M pop</th>
 <th width="30">New Deaths/1M pop</th>
 <th width="30">Active Cases/1M pop</th>
 </tr>]

In [39]:
headings = []
for th in head[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    print(th.text)
    #headings.append(td.b.text.replace('\n', ' ').strip())
    headings.append(th.text.replace("\n","").strip())
print(headings)

#
Country,Other
TotalCases
NewCases
TotalDeaths
NewDeaths
TotalRecovered
NewRecovered
ActiveCases
Serious,Critical
Tot Cases/1M pop
Deaths/1M pop
TotalTests
Tests/
1M pop

Population
Continent
1 Caseevery X ppl
1 Deathevery X ppl
1 Testevery X ppl
New Cases/1M pop
New Deaths/1M pop
Active Cases/1M pop
['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'Tot\xa0Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop', 'Population', 'Continent', '1 Caseevery X ppl', '1 Deathevery X ppl', '1 Testevery X ppl', 'New Cases/1M pop', 'New Deaths/1M pop', 'Active Cases/1M pop']


In [40]:
body = covid_table.tbody.find_all("tr") 
body[0] #here is one example of HTML snippet for one row

<tr class="total_row_world row_continent" data-continent="North America" style="display: none">
<td></td>
<td style="text-align:left;">
<nobr>North America</nobr>
</td>
<td>62,971,994</td>
<td>+3,563</td>
<td>1,234,842</td>
<td>+162</td>
<td>49,685,595</td>
<td>+2,006</td>
<td>12,051,557</td>
<td>21,837</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td data-continent="North America" style="display:none;">North America</td>
<td>
</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>

In [41]:
#lets declare empty list data that will hold all rows data
data = []
for r in range(1,len(body)):
    row = [] # empty lsit to hold one row data
    for tr in body[r].find_all("td"):
        row.append(tr.text.replace("\n","").strip())
        #append row data to row after removing newlines escape and triming unnecesary spaces
    data.append(row)
    
# data contains all the rows excluding header
# row contains data for one row

In [42]:
#We can now pass data into a pandas dataframe
#with headings as the columns
df = pd.DataFrame(data,columns=headings)
df.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",...,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,,Asia,"84,024,163","+12,968","1,247,348",+373,"81,347,410","+18,986","1,429,405","29,707",...,,,,Asia,,,,,,
1,,South America,"39,432,897","+2,150","1,190,369",+14,"37,389,799","+1,462","852,729","12,673",...,,,,South America,,,,,,
2,,Europe,"83,010,451","+40,610","1,506,463","+1,328","72,463,266","+106,638","9,040,722","22,676",...,,,,Europe,,,,,,
3,,Africa,"9,503,698",+143,"227,500",,"8,522,657",+6,"753,541","1,708",...,,,,Africa,,,,,,
4,,Oceania,"454,631","+9,943","4,447",+6,"374,065",,"76,119",170,...,,,,Australia/Oceania,,,,,,


In [43]:
data = df[df["#"]!=""].reset_index(drop=True)
# Data points with # value are the countries of the world while the data points with
# null values for # columns are features like continents totals etc
data = data.drop_duplicates(subset = ["Country,Other"])
#Reason to drop duplicates : Worldometer reports data for 3 days: today and 2 days back
#I found out that removing duplicates removes the values for the bast two days and keep today's

In [44]:
#We can drop the following columns - Opinion
cols = ['#',
 'Tot\xa0Cases/1M pop',
 'Deaths/1M pop',
 'Tests/1M pop',
 'Population',
 '1 Caseevery X ppl',
 '1 Deathevery X ppl',
 '1 Testevery X ppl']

In [45]:
data_final = data.drop(cols,axis=1)
data_final.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalTests,Continent,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,USA,"52,986,307",,"837,671",,"40,994,250",,"11,154,386","16,167","798,461,897",North America,,,"33,409"
1,India,"34,779,815",,"479,520",,"34,223,263","+7,286","77,032","8,944","671,051,627",Asia,,,55
2,Brazil,"22,230,737",,"618,429",,"21,414,318",,"197,990","8,318","63,776,166",South America,,,922
3,UK,"11,891,292",,"147,857",,"9,961,369",,"1,782,066",842,"395,579,689",Europe,,,"26,049"
4,Russia,"10,368,299","+24,946","303,250",+981,"9,222,486","+42,107","842,563","2,300","237,800,000",Europe,171,7,"5,770"


In [12]:
#Once you are happy then you can save the dataframe as csv or xlsx
#data_final.to_csv("covid.csv",index=False)

In [13]:
#Here is a list of continents
list(data_final["Continent"].unique())

['North America',
 'Asia',
 'South America',
 'Europe',
 'Africa',
 'Australia/Oceania',
 '']

In [19]:
#Then you can filter by continent
data_final[data_final["Continent"]=="Europe"].head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalTests,Continent,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
3,UK,"11,891,292",,"147,857",,"9,961,369",,"1,782,066",842,"395,579,689",Europe,,,"26,049"
4,Russia,"10,368,299","+24,946","303,250",+981,"9,222,486","+42,107","842,563","2,300","237,800,000",Europe,171,7,"5,770"
6,France,"8,983,760",,"122,462",,"7,752,550",,"1,108,748","3,254","182,873,794",Europe,,,"16,931"
7,Germany,"6,982,228",,"110,908",,"6,087,300","+38,500","784,020","4,636","89,622,218",Europe,,,"9,314"
9,Spain,"5,718,007",,"89,019",,"5,002,310",,"626,678","1,515","66,213,858",Europe,,,"13,396"


## METHOD 2 : USING XPath

In [20]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

In [21]:
#Rememeber that rows on HTML are stored between <tr>..</tr> tags
url = "https://www.worldometers.info/coronavirus/"
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr') #note that we are only using tr because we have only one table on the site
#if we had many then we could have needed to have the XPath so that we are specific.
#XPath can be accessed by inspecting elements of a site. 
print(len(list(tr_elements))) #number of all rows including the header row

723


In [22]:
#Lets deal with the header row first
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))


1:"#"
2:"Country,Other"
3:"TotalCases"
4:"NewCases"
5:"TotalDeaths"
6:"NewDeaths"
7:"TotalRecovered"
8:"NewRecovered"
9:"ActiveCases"
10:"Serious,Critical"
11:"Tot Cases/1M pop"
12:"Deaths/1M pop"
13:"TotalTests"
14:"Tests/
1M pop
"
15:"Population"
16:"Continent"
17:"1 Caseevery X ppl"
18:"1 Deathevery X ppl"
19:"1 Testevery X ppl"
20:"New Cases/1M pop"
21:"New Deaths/1M pop"
22:"Active Cases/1M pop"


In [33]:
# Since out first row is the header, data is stored on the second row onwards
for j in range(1, len(tr_elements)):
    # T is our j'th row
    T = tr_elements[j]
    # If row is not of size 22, the //tr data is not from our table
    if len(T) != 22: # disadvantage of this method: this can change if the website is changed.
        break

    # i is the index of our column
    i = 0

    # Iterate through each element of the row
    for t in T.iterchildren():
        data = t.text_content()
        # Check if row is empty
        if i > 0:
            # Convert any numerical value to integers
            try:
                data = int(data)
            except:
                pass
        # Append the data to the empty list of the i'th column
        col[i][1].append(data)
        # Increment i for the next column
        i += 1

#ideally all columns must have the same number of rows
print([len(C) for (title,C) in col]) #for this case we have 1444

[1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444, 1444]


In [31]:
#From here the same explanation holds like in method 1 above
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)
data = df[df["#"]!=""].reset_index(drop=True)
data = data.drop_duplicates(subset = ["Country,Other"])

In [32]:
cols = ['#',
 'Tot\xa0Cases/1M pop',
 'Deaths/1M pop',
 'Tests/\n1M pop\n',
 'Population',
 'Continent',
 '1 Caseevery X ppl',
 '1 Deathevery X ppl',
 '1 Testevery X ppl']

data_final = data.drop(cols,axis=1)
#Assume we want to pick only these 5 countries
c = ["USA" , "Germany","Belgium","Kenya","Cameroon"]
data = data_final[data_final["Country,Other"].isin(c)].reset_index(drop=True)
data.head(10)

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalTests,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,USA,"52,986,307",,"837,671",,"40,994,250",,"11,154,386","16,167","798,461,897",,,"33,409"
1,Germany,"6,982,228",,"110,908",,"6,087,300","+38,500","784,020","4,636","89,622,218",,,"9,314"
2,Belgium,"2,038,111","+7,233","28,149",39,"1,595,493",,"414,469",660,"26,828,439",620,3,"35,534"
3,Kenya,"277,609",,"5,357",,"249,694",,"22,558",16,"2,975,795",,,406
4,Cameroon,"108,451",,"1,851",,"105,735",,865,13,"1,751,774",,,31
